What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [2]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging

  Activating project at `~/Globtim.jl`


In [3]:
# Constants and Parameters
const n, a, b = 2, 11, 10
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = Deuflhard # Objective function

Deuflhard (generic function with 1 method)

One may assume that when we have access to exact evaluations, we would want to have a small $L^2$-norm tolerance `tol_l2 = 5e-4` and high probability of computing an accurate discrete $L^2$-norm `alpha= 1/10`.

We need to also return the number of samples used to generate the sample set. It is annoying that the error goes up while the degree has increased.

In [4]:
d = 8 # Initial Degree 
SMPL = 40 # Number of samples
center = [0.0, 0.0]
TR = test_input(n, center, (alpha, delta), tol_l2, (0.0, 0.0), scale_factor, 1.0, f)
pol_cheb = Constructor(TR, d, GN=SMPL, basis=:chebyshev)
pol_lege = Constructor(TR, d, GN=SMPL, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.009384317330376233
Number of samples: 40
current L2-norm: 0.009679838314339191
Number of samples: 40


In [5]:
msolve_polynomial_system(pol_cheb, x)

Input file already exists: inputs.ms
Output file already exists: outputs.ms

---------------- TIMINGS ----------------
overall(elapsed)        0.00 sec
overall(cpu)            0.00 sec
select                  0.00 sec   7.7%
symbolic prep.          0.00 sec  15.0%
update                  0.00 sec  16.7%
convert                 0.00 sec  22.5%
linear algebra          0.00 sec  12.5%
reduce gb               0.00 sec   0.0%
-----------------------------------------

---------- COMPUTATIONAL DATA -----------
size of basis                     8
#terms in basis                 350
#pairs reduced                    8
#GM criterion                    28
#redundant elements               1
#rows reduced                    24
#zero reductions                  1
max. matrix data                 17 x 65 (69.321%)
max. symbolic hash table size  2^11
max. basis hash table size     2^16
-----------------------------------------


---------------- TIMINGS ----------------
overall(elapsed)        0.00 


--------------- INPUT DATA ---------------
#variables                       2
#equations                       2
#invalid equations               0
field characteristic             0
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                         1
info level                       1
generate pbm files               0
------------------------------------------



<Step:16/0.00/0.01>
------------------------------------------------------------------------------------------------------

---------- COMPUTATIONAL DATA -----------
#primes                         119
#bad primes                       0
-----------------------------------------

---------------- TIMINGS ----------------
CRT and ratrecon(elapsed)       0.00 sec
-----------------------------------------

---------------- TIMINGS ----------------
rational param(elapsed)         0.15 sec
rational param(cpu)             0.14 sec
-----------------------------------------


[0][1]Real root isolation starts at precision 174
Number of real roots: 9
Starts real root extraction.
Elapsed time (real root extraction) = 0.00


------------------------------------------------------------------------------------
msolve overall time           0.15 sec (elapsed) /  0.14 sec (cpu)
------------------------------------------------------------------------------------


Process(`msolve -v 1 -f inputs.ms -o outputs.ms`, ProcessExited(0))

Solve the system of partial derivatives using `Msolve`. 

In [6]:
df_cheb = msolve_parser("outputs.ms", f, scale_factor, n)

Row,x1,x2,z
,Float64,Float64,Float64
1,0.316356,-0.998837,0.0424662
2,0.780329,-0.780329,0.144272
3,-0.39892,-0.39892,2.65526
4,0.998837,-0.316356,0.0424662
5,1.04917e-15,-4.33605e-15,4.0
6,-0.998837,0.316356,0.0424662
7,0.39892,0.39892,2.65526
8,-0.780329,0.780329,0.144272
9,-0.316356,0.998837,0.0424662


In [7]:
using GLMakie

# Extract coordinates and function values
# Change the coordinates to uniform grid
coords = pol_cheb.scale_factor * pol_cheb.grid
z_coords = pol_cheb.z

if size(coords)[2] == 2  # Plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Deuflhard Function",
        xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Scale the domain
    scale_factor = pol_cheb.scale_factor  # Using the scale factor from your polynomial

    # Define threshold for switching between scatter and surface
    point_threshold = 1000  # Adjust this value based on your needs
    sample_fraction = 0.2   # Fraction of points to use if exceeding threshold

    if length(z_coords) > point_threshold
        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Determine grid dimensions
        nx = length(x_unique)
        ny = length(y_unique)

        # Reshape data into a grid
        z_grid = reshape(z_coords, nx, ny)

        # Create surface plot with scaled domain
        surface!(ax, scale_factor .* x_unique, scale_factor .* y_unique, z_grid,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Sample a fraction of points for scatter overlay
        n_points = length(z_coords)
        sample_indices = rand(1:n_points, Int(floor(n_points * sample_fraction)))

        # Plot sampled scatter points with scaled domain
        scatter!(ax, scale_factor .* coords[sample_indices, 1],
            scale_factor .* coords[sample_indices, 2],
            z_coords[sample_indices],
            markersize=2,
            color=:black,
            label="Sampled Data Points")
    else
        # Original scatter plot for smaller datasets with scaled domain
        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=2,
            color=:black,
            label="Sampled Data Core")

        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=4,
            color=z_coords,
            colormap=:viridis,
            label="Sampled Data Halo")
    end

    # Plot the critical points with scaled domain
    scatter!(ax, scale_factor .* df_cheb.x1,
        scale_factor .* df_cheb.x2,
        df_cheb.z,
        markersize=10,
        color=:orange,
        label="Chebyshev approximant critical points")

    # scatter!(ax, scale_factor .* df_lege.x1,
    #     scale_factor .* df_lege.x2,
    #     df_lege.z,
    #     markersize=10,
    #     color=:yellow,
    #     label="Legendre approximant critical points")

    display(fig)
end

2024-12-10 18:53:14.983 julia[33160:2510561] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-10 18:53:14.983 julia[33160:2510561] +[IMKInputSession subclass]: chose IMKInputSession_Modern


GLMakie.Screen(...)